In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from vmtk import pypes
from vmtk import vmtkscripts


base_dir = os.path.abspath('/media/tianyi/data/Analysis/DCM2BIDS5/')
vtp_dir = os.path.join(base_dir, 'derivatives', 'manualwork', 'segmentations', 'veins_superficial', 'SSS-LTS-RTS')
output_dir = os.path.join(base_dir, 'derivatives', 'manualwork', 'vmtk_superficial-veins', 'vtp-from-Slicer')


In [2]:
vtp
# centerline reading
centerlineReader = vmtkscripts.vmtkSurfaceReader()
centerlineReader.InputFileName = outCenterline
centerlineReader.Execute()
clNumpyAdaptor = vmtkscripts.vmtkCenterlinesToNumpy()
clNumpyAdaptor.Centerlines = centerlineReader.Surface
clNumpyAdaptor.Execute()
numpyCenterlines = clNumpyAdaptor.ArrayDict

Reading VTK XML surface file.
wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
Radius
EdgeArray
EdgePCoordArray
converting cell connectivity list


In [4]:
def centerline_extractor(outCenterline):

    # centerline reading
    centerlineReader = vmtkscripts.vmtkSurfaceReader()
    centerlineReader.InputFileName = outCenterline
    centerlineReader.Execute()
    clNumpyAdaptor = vmtkscripts.vmtkCenterlinesToNumpy()
    clNumpyAdaptor.Centerlines = centerlineReader.Surface
    clNumpyAdaptor.Execute()
    numpyCenterlines = clNumpyAdaptor.ArrayDict

    points = numpyCenterlines['Points']
    pointsIds = np.arange(len(points))
    radius = numpyCenterlines['PointData']['MaximumInscribedSphereRadius']
    segPointIds = numpyCenterlines['CellData']['CellPointIds']
    numSegments = len(segPointIds)

    frames = []
    for segId in range(numSegments):
        segment = segPointIds[segId]
        segData = pd.DataFrame({'pointId':segment,'segId':segId*np.ones(len(segment), dtype=int),'x':points[segment,0],'y':points[segment,1],'z':points[segment,2],'radius':radius[segment]})
        frames.append(segData)    
    CenterlineData = pd.concat(frames)


    for segId in range(numSegments):
        segment = CenterlineData[CenterlineData['segId'] == segId]

        x = np.array(segment['x'])
        y = np.array(segment['y'])
        z = np.array(segment['z'])

        diff_x = np.diff(x)
        diff_y = np.diff(y)
        diff_z = np.diff(z)

        steplength = np.sqrt(np.square(diff_x) + np.square(diff_y) + np.square(diff_z))
        distance = np.cumsum(steplength)
        distance = np.insert(distance,0,0)

        CenterlineData.loc[CenterlineData['segId'] == segId, 'distance'] = distance

    CenterlineData['diameter'] = 2 * CenterlineData['radius']

    return CenterlineData

In [5]:
centerline_df = centerline_extractor(vtp_fn)

Reading VTK XML surface file.
wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
Radius
EdgeArray
EdgePCoordArray
converting cell connectivity list


TypeError: unhashable type: 'numpy.ndarray'